In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

from torch import nn,optim
import torch

import algo
import scripts
import attack

import matplotlib.pyplot as plt
import json 

import os

import random



2023-01-19 13:56:54.835414: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# purchase
rand_seed=42
np.random.seed(rand_seed)
torch.manual_seed(rand_seed)
random.seed(rand_seed)

x_target_train = np.load('data/rs'+str(rand_seed)+'_x_target_train.npy')
y_target_train = np.load('data/rs'+str(rand_seed)+'_y_target_train.npy')
x_target_test = np.load('data/rs'+str(rand_seed)+'_x_target_test.npy')
y_target_test = np.load('data/rs'+str(rand_seed)+'_y_target_test.npy')
n_classes = len(np.unique(y_target_train))
X_train_size = x_target_train.shape[0]
X_test_size = x_target_test.shape[0]


In [3]:
# X_train_size = 5000
# X_test_size = 5000
# x_target_train = np.array(x_target_train [:X_train_size])
# y_target_train = np.array(y_target_train [:X_train_size])
# x_target_train.shape

(5000, 600)

In [ ]:
model = algo.LogisticRegression_DPSGD()

model.n_classes      = len(np.unique(y_target_test))
model.alpha          = 0.01
model.max_iter       = 100
model.lambda_        = 1e-5
model.tolerance      = 1e-5
model.DP             = False
model.L              = 1
model.epsilon        = 0

In [ ]:
rand_seed=42
np.random.seed(rand_seed)
torch.manual_seed(rand_seed)
random.seed(rand_seed)

X,y = model.init_theta(x_target_train, y_target_train)
model.train(X,y)


In [ ]:
model.evaluate(x_target_train, y_target_train, acc=True)
model.evaluate(x_target_test, y_target_test, acc=True)


In [ ]:
epsilon_out_dp = 100
noise_model = scripts.output_DP(target_model, x_target_train.shape[0], epsilon_out_dp)
noise_model.train_acc = noise_model.evaluate(x_target_train, y_target_train)
noise_model.test_acc = noise_model.evaluate(x_target_test, y_target_test)
noise_model.mi_attack = attack.test_mi_attack(ams, model, x_target_train, y_target_train, x_target_test, y_target_test)
noise_model.out_dp_mi_attack = attack.test_mi_attack(ams, noise_model, x_target_train, y_target_train, x_target_test, y_target_test)

In [ ]:
noise_model.__dict__

# Centralized output DP

In [4]:
# purchase
# load data with different seeds
data_seed = {}
for rand_seed in [1,3,13,24,42]:
    data_seed[rand_seed] =  {}
    
    data_seed[rand_seed]['x_target_train'] = np.load('data/rs'+str(rand_seed)+'_x_target_train.npy')
    data_seed[rand_seed]['y_target_train'] = np.load('data/rs'+str(rand_seed)+'_y_target_train.npy')
    data_seed[rand_seed]['x_target_test'] = np.load('data/rs'+str(rand_seed)+'_x_target_test.npy')
    data_seed[rand_seed]['y_target_test'] = np.load('data/rs'+str(rand_seed)+'_y_target_test.npy')
    data_seed[rand_seed]['n_classes'] = len(np.unique(y_target_train))
    data_seed[rand_seed]['X_train_size'] = x_target_train.shape[0]
    data_seed[rand_seed]['X_test_size ']= x_target_test.shape[0]

def set_the_seed_and_data(seed):
    np.random.seed(rand_seed)
    torch.manual_seed(rand_seed)
    random.seed(rand_seed)
    
    return data_seed[seed]['x_target_train'], data_seed[rand_seed]['y_target_train'], data_seed[rand_seed]['x_target_test'], data_seed[rand_seed]['y_target_test']

In [5]:
# attack models
from torch import nn

class Net_attack(nn.Module):

    def __init__(self, h_neurons, do, input_size):
        super(Net_attack, self).__init__()
        self.input_size = input_size
        self.h_neurons = h_neurons
        self.do = do
        self.fc1 = nn.Linear(input_size, h_neurons)
        self.fc2 = nn.Linear(h_neurons, 2)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(do)
        self.softmax = nn.Softmax(dim=1)   

    def forward(self, x):
        x = x.view(-1, self.input_size)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.drop(x)
        x = self.softmax(x)
        return x

path = 'mia'
ams = {}
for r,d,f in os.walk(path):
    for file in f:
        if "best_mi_model" in file:
            ams[file] = Net_attack(h_neurons=64, do=0, input_size=200)
            ams[file] = torch.load(r+'/'+file)      

In [6]:
path = 'tm'
tms_params = {}
tms = {}
for r,d,f in os.walk(path):
    for file in f:
        if "target_model_params.json" in file:
            with open(path+'/'+file) as json_file:
                tms_params[file.replace('_params.json', '')] = json.load(json_file)
        if "target_model.npy" in file:
            tms[file.replace('.npy', '')] = np.load(path+'/'+file)
            
df = pd.DataFrame.from_dict(tms_params, orient='index')
df.shape            

(668, 21)

In [ ]:
#Load model and parameters
file = 'tm/rs42_lr0.001_iter100_reg0.005_DPFalse_target_model_params.json'
with open(file) as json_file:
    tm_params = json.load(json_file)
            
# x_target_train, y_target_train, x_target_test, y_target_test = attack.data_shuffle(rand_seed, X_raw, y_raw)
target_model = algo.LogisticRegression_DPSGD()
scripts.set_model_params(target_model, tm_params)
target_model.theta = np.load('tm/rs42_lr0.001_iter100_reg0.005_DPFalse_target_model.npy')
target_model.evaluate(x_target_train, y_target_train, acc=True)
target_model.evaluate(x_target_test, y_target_test, acc=True)

#Output DP
noise_model = scripts.output_DP(target_model, epsilon_out_DP=10, delta_out_DP=1e-5, X_train_size=X_train_size)
noise_model.train_acc = noise_model.evaluate(x_target_train, y_target_train, acc=True)
noise_model.test_acc = noise_model.evaluate(x_target_test, y_target_test, acc=True)
    
    

# FL output DP

In [43]:
# purchase
rand_seed=42
np.random.seed(rand_seed)
torch.manual_seed(rand_seed)
random.seed(rand_seed)

x_target_train = np.load('data/rs'+str(rand_seed)+'_x_target_train.npy')
y_target_train = np.load('data/rs'+str(rand_seed)+'_y_target_train.npy')
x_target_test = np.load('data/rs'+str(rand_seed)+'_x_target_test.npy')
y_target_test = np.load('data/rs'+str(rand_seed)+'_y_target_test.npy')
n_classes = len(np.unique(y_target_train))
X_train_size = x_target_train.shape[0]
X_test_size = x_target_test.shape[0]


In [112]:
path = 'fl'
params = {}
results = {}
models = {}
ouput_dp_results = {}
for r,d,f in os.walk(path):
    for file in f:
        if "params.json" in file:
            with open(r+'/'+file) as json_file:
                params[r] = json.load(json_file)
        elif "results.csv" in file:
            results[r] = pd.read_csv(r+'/'+file)
            it = []
            client = []
            for k in results[r]['Unnamed: 0']:
                it.append(k[k.find('i')+1:k.find('_')])
                client.append(k[k.find('_')+1:])
            results[r]['it'] = it
            results[r]['client'] = client
        elif '.npy' in file:
            if r not in models:
                models[r] = {}
            models[r][file] = np.load(r+'/'+file)
        elif 'outputDP.json' in file:
            with open(r+'/'+file) as json_file:
                ouput_dp_results[r+'/'+file] = json.load(json_file)



In [155]:
fl_file = 'fl/rs42_ncl2_fiter5_lr0.01_iter100_reg0.0001_DPFalse'
print(results[fl_file])
params[fl_file]

   Unnamed: 0       0       1 it client
0       i0_c0  1.0000  0.4353  0     c0
1       i0_c1  1.0000  0.4372  0     c1
2        i0_g  0.9745  0.5214  0      g
3       i1_c0  1.0000  0.4394  1     c0
4       i1_c1  1.0000  0.4373  1     c1
5        i1_g  0.9771  0.5243  1      g
6       i2_c0  1.0000  0.4368  2     c0
7       i2_c1  1.0000  0.4379  2     c1
8        i2_g  0.9769  0.5211  2      g
9       i3_c0  1.0000  0.4362  3     c0
10      i3_c1  1.0000  0.4375  3     c1
11       i3_g  0.9749  0.5206  3      g
12      i4_c0  1.0000  0.4372  4     c0
13      i4_c1  1.0000  0.4376  4     c1
14       i4_g  0.9762  0.5219  4      g


{'n_classes': 100,
 'alpha': 0.01,
 'max_iter': 100,
 'lambda_': 0.0001,
 'tolerance': 1e-05,
 'DP': False,
 'L': 1,
 'C': 1,
 'epsilon': 0,
 'delta': 1e-05}

In [156]:
if os.path.exists(fl_file+f'/results_outDP.json'):
    print('file exists')


In [162]:
np.random.seed(rand_seed)
torch.manual_seed(rand_seed)
random.seed(rand_seed)

vals = ['epsilon_out', 'train_acc', 'test_acc', 'out_dp_train_acc', 'out_dp_test_acc', 'mi_attack', 'out_dp_mi_attack']
epses = [i/10 for i in range(1,10)] + [i for i in range(1,11)] + [i*10 for i in range(2,11)] + [i*100 for i in range(2,11)]+ [i*1000 for i in range(2,11)] + [i*10000 for i in range(2,11)]
# epses = [1,5,10]

number_of_clients = len(results[fl_file]['client'].unique())-1
data_per_client = int(x_target_train.shape[0]/number_of_clients)
clients = results[fl_file]['client'].unique()
fl0 = {}
fl0_out_dp = {}
fl_iteration = 2

out_eps_results = {}
for epsilon_out_dp in epses:
    out_eps_results[epsilon_out_dp] = {}
#     if os.path.exists(fl_file+f'/i{fl_iteration}_{c}_outputDP.json'):
#         print("file already exists")
#         continue
    for i,c in enumerate(clients):
        fl0[c] = algo.LogisticRegression_DPSGD()
        scripts.set_model_params(fl0[c], params[fl_file])
        if c == 'g':
            fl0[c].x = x_target_train
            fl0[c].y = y_target_train
            fl0[c].theta = models[fl_file][f'i{fl_iteration}_{c}.npy']
            fl0_out_dp[c] = algo.LogisticRegression_DPSGD()
            scripts.set_model_params(fl0_out_dp[c], fl0_out_dp['c0'].__dict__)
            fl0_out_dp[c].theta = sum([fl0_out_dp[c].theta for c in fl0_out_dp if c!='g'])/number_of_clients
        else:
            fl0[c].x = x_target_train[i*data_per_client:(i+1)*data_per_client]
            fl0[c].y = y_target_train[i*data_per_client:(i+1)*data_per_client]
            fl0[c].theta = models[fl_file][f'i{fl_iteration}_{c}.npy']
            fl0_out_dp[c] = scripts.output_DP(fl0[c], fl0[c].x.shape[0], epsilon_out_dp)
        fl0_out_dp[c].train_acc = fl0[c].evaluate(fl0[c].x, fl0[c].y)
        fl0_out_dp[c].test_acc = fl0[c].evaluate(x_target_test, y_target_test)
        fl0_out_dp[c].out_dp_train_acc = fl0_out_dp[c].evaluate(fl0[c].x, fl0[c].y)
        fl0_out_dp[c].out_dp_test_acc = fl0_out_dp[c].evaluate(x_target_test, y_target_test)
        fl0_out_dp[c].mi_attack = attack.test_mi_attack(ams, fl0[c], fl0[c].x, fl0[c].y, x_target_test, y_target_test)
        fl0_out_dp[c].out_dp_mi_attack = attack.test_mi_attack(ams, fl0_out_dp[c], fl0[c].x, fl0[c].y, x_target_test, y_target_test)
        
        temp_d = dict(fl0_out_dp[c].__dict__)
        out_eps_results[epsilon_out_dp][c] = {k: v for k, v in temp_d.items() if k in vals}
   

In [163]:
if os.path.exists(fl_file+f'/fliter{fl_iteration}_results_outDP.json'):
    print('file exists')
else:
    with open(fl_file+f'/fliter{fl_iteration}_results_outDP.json', 'w') as file:
            json.dump(out_eps_results, file)

In [164]:
fl_file+f'/fliter{fl_iteration}_results_outDP.json'

'fl/rs42_ncl2_fiter5_lr0.01_iter100_reg0.0001_DPFalse/fliter2_results_outDP.json'

In [3]:
'fl/rs42_ncl2_fiter5_lr0.01_iter100_reg0.0001_DPFalse/fliter2_results_outDP.json'


1

In [ ]:
# output dp with fixed epsilon save files

In [ ]:
np.random.seed(rand_seed)
torch.manual_seed(rand_seed)
random.seed(rand_seed)


epsilon_out_dp = 100

number_of_clients = len(results[fl_file]['client'].unique())-1
data_per_client = int(x_target_train.shape[0]/number_of_clients)
clients = results[fl_file]['client'].unique()
fl0 = {}
fl0_out_dp = {}
fl_iteration = 0
for fl_iteration in [0,1,2,3,4]:
    if os.path.exists(fl_file+f'/i{fl_iteration}_{c}_outputDP.json'):
#         print("file already exists")
        continue
    for i,c in enumerate(clients):
        fl0[c] = algo.LogisticRegression_DPSGD()
        scripts.set_model_params(fl0[c], params[fl_file])
        if c == 'g':
            fl0[c].x = x_target_train
            fl0[c].y = y_target_train
            fl0[c].theta = models[fl_file][f'i{fl_iteration}_{c}.npy']
            fl0_out_dp[c] = algo.LogisticRegression_DPSGD()
            scripts.set_model_params(fl0_out_dp[c], fl0_out_dp['c0'].__dict__)
            fl0_out_dp[c].theta = sum([fl0_out_dp[c].theta for c in fl0_out_dp if c!='g'])/number_of_clients
        else:
            fl0[c].x = x_target_train[i*data_per_client:(i+1)*data_per_client]
            fl0[c].y = y_target_train[i*data_per_client:(i+1)*data_per_client]
            fl0[c].theta = models[fl_file][f'i{fl_iteration}_{c}.npy']
            fl0_out_dp[c] = scripts.output_DP(fl0[c], fl0[c].x.shape[0], epsilon_out_dp)
        fl0_out_dp[c].train_acc = fl0[c].evaluate(fl0[c].x, fl0[c].y)
        fl0_out_dp[c].test_acc = fl0[c].evaluate(x_target_test, y_target_test)
        fl0_out_dp[c].out_dp_train_acc = fl0_out_dp[c].evaluate(fl0[c].x, fl0[c].y)
        fl0_out_dp[c].out_dp_test_acc = fl0_out_dp[c].evaluate(x_target_test, y_target_test)
        fl0_out_dp[c].mi_attack = attack.test_mi_attack(ams, fl0[c], fl0[c].x, fl0[c].y, x_target_test, y_target_test)
        fl0_out_dp[c].out_dp_mi_attack = attack.test_mi_attack(ams, fl0_out_dp[c], fl0[c].x, fl0[c].y, x_target_test, y_target_test)

#     np.save(fl_file+f'/i{fl_iteration}_{c}_outputDP.npy', fl0_out_dp[c].theta)
    temp_d = dict(fl0_out_dp[c].__dict__)
    temp_d.pop('theta')
    with open(fl_file+f'/i{fl_iteration}_{c}_outputDP.json', 'w') as file:
        json.dump(temp_d, file)